In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cnpj_final = pd.read_csv('/content/drive/MyDrive/TCC - PUC/cnpj_final.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)

In [4]:
municipios_siafi = pd.read_csv('//content/drive/MyDrive/TCC - PUC/TABELA DE MUNICÍPIOS DO SIAFI.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [5]:
municipios_siafi.columns = ['codigo_municipio_siafi', 'cnpj_base', 'descricao_municipio', 'uf','codigo_ibge']

In [6]:
codigo_ibge_ajustado = municipios_siafi['codigo_ibge'].map(lambda x: str(x)[:-1])

In [7]:
municipios_siafi = pd.concat([municipios_siafi, codigo_ibge_ajustado], axis=1)

In [8]:
municipios_siafi['codigo_municipio_siafi'] = municipios_siafi['codigo_municipio_siafi'].astype(int)
cnpj_final['codigo_municipio_siafi'] = cnpj_final['codigo_municipio_siafi'].astype(int)

In [9]:
municipios_siafi.columns = ['codigo_municipio_siafi', 'cnpj_base', 'descricao_municipio', 'uf','codigo_ibge', 'codigo_ibge_ajustado']

In [10]:
municipios_siafi = municipios_siafi.drop(['cnpj_base','uf'], axis=1) #eliminação das colunas desnecessárias

In [11]:
cnpj_final = pd.merge(cnpj_final, municipios_siafi, how='left', on='codigo_municipio_siafi')

In [12]:
codigo_ibge_data_fechamento = pd.DataFrame (np.where((cnpj_final['data_situacao_cadastral'] >= '20200401') & (cnpj_final['codigo_situacao_cadastral']=='8'), cnpj_final['data_situacao_cadastral'] + cnpj_final['codigo_ibge_ajustado'] , "19800101" + cnpj_final['codigo_ibge_ajustado']))

In [13]:
codigo_ibge_data_abertura = pd.DataFrame (np.where((cnpj_final['data_situacao_cadastral'] >= '20200401') & (cnpj_final['codigo_situacao_cadastral']=='8'), "19800101" + cnpj_final['codigo_ibge_ajustado'], "20210531" + cnpj_final['codigo_ibge_ajustado']))

In [14]:
codigo_ibge_data_todos = pd.DataFrame ("20210531" + cnpj_final['codigo_ibge_ajustado'])

In [15]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_fechamento, codigo_ibge_data_abertura], axis=1)

In [16]:
cnpj_final.columns = ['cnpj_basico','cnpj_ordem','cnpj_dv','matriz_filial','codigo_situacao_cadastral','data_situacao_cadastral','motivo_situacao_cadastral','data_inicio_atividade','cnae_principal','codigo_municipio_siafi','capital_social','descricao_municipio','codigo_ibge','codigo_ibge_ajustado','codigo_ibge_data_fechamento','codigo_ibge_data_abertura']

In [17]:
covid_brasil_1 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_2020_Parte1_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)
covid_brasil_2 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_2020_Parte2_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)
covid_brasil_3 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_Parte3_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)

In [18]:
covid_brasil_todas = pd.concat([covid_brasil_1, covid_brasil_2, covid_brasil_3], axis=0)

In [19]:
covid_brasil_todas = covid_brasil_todas[covid_brasil_todas['codmun'] .notnull()] 
covid_brasil_todas = covid_brasil_todas[covid_brasil_todas['municipio'] .notnull()] 

In [20]:
covid_brasil_todas = covid_brasil_todas.drop(['municipio','coduf','codRegiaoSaude','nomeRegiaoSaude','semanaEpi','Recuperadosnovos','emAcompanhamentoNovos','emAcompanhamentoNovos', 'interior/metropolitana'], axis=1) #eliminação das colunas desnecessárias

In [21]:
covid_brasil_todas.columns = ['regiao', 'estado', 'codigo_ibge_ajustado', 'data_covid','populacaoTCU2019', 'casosAcumulado','casosNovos','obitosAcumulado','obitosNovos']

In [22]:
data_removido = covid_brasil_todas['data_covid'].str.replace(r'\D', '')

In [23]:
codigo_ibge_data = data_removido + covid_brasil_todas['codigo_ibge_ajustado']

In [24]:
covid_brasil_todas['populacaoTCU2019'] = covid_brasil_todas['populacaoTCU2019'].astype(int)
covid_brasil_todas['casosAcumulado'] = covid_brasil_todas['casosAcumulado'].astype(int)
covid_brasil_todas['obitosAcumulado'] = covid_brasil_todas['obitosAcumulado'].astype(int)

In [25]:
casos_acumulados_porcentagem = round(((covid_brasil_todas['casosAcumulado'] * 100) / (covid_brasil_todas['populacaoTCU2019'])),10)
obitos_acumulados_porcentagem = round(((covid_brasil_todas['obitosAcumulado'] * 100) / (covid_brasil_todas['populacaoTCU2019'])),10)

In [26]:
covid_brasil_todas = pd.concat([covid_brasil_todas, casos_acumulados_porcentagem, obitos_acumulados_porcentagem, codigo_ibge_data], axis=1)

In [27]:
covid_brasil_todas.columns = ['regiao', 'estado', 'codigo_ibge_ajustado','data_covid', 'populacaoTCU2019','casosAcumulado', 'casosNovos','obitosAcumulado','obitosNovos', 'casos_acumulados_porcentagem','obitos_acumulados_porcentagem','codigo_ibge_data']

In [28]:
codigo_ibge_data_todos.columns = ['codigo_ibge_data']

In [29]:
codigo_ibge_data_todos  = pd.merge(codigo_ibge_data_todos, covid_brasil_todas, how='left', on='codigo_ibge_data')

In [30]:
codigo_ibge_data_todos = codigo_ibge_data_todos.drop(['codigo_ibge_data','codigo_ibge_ajustado','data_covid','casosAcumulado','casosNovos','obitosAcumulado','obitosNovos','casos_acumulados_porcentagem','obitos_acumulados_porcentagem'], axis=1) #eliminação das colunas desnecessárias

In [31]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_todos], axis=1)

In [32]:
codigo_ibge_data_fechamento.columns = ['codigo_ibge_data']

In [33]:
codigo_ibge_data_fechamento  = pd.merge(codigo_ibge_data_fechamento, covid_brasil_todas, how='left', on='codigo_ibge_data')

In [34]:
codigo_ibge_data_fechamento = codigo_ibge_data_fechamento.drop(['codigo_ibge_data','regiao','estado','codigo_ibge_ajustado','data_covid','populacaoTCU2019','casosNovos','obitosNovos'], axis=1) #eliminação das colunas desnecessárias

In [35]:
codigo_ibge_data_fechamento.columns = ['casosAcumulado_covid','obitosAcumulado_covid','casos_acumulados_porcentagem_covid','obitos_acumulados_porcentagem_covid']

In [36]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_fechamento], axis=1)

In [37]:
codigo_ibge_data_abertura.columns = ['codigo_ibge_data']

In [38]:
codigo_ibge_data_abertura  = pd.merge(codigo_ibge_data_abertura, covid_brasil_todas, how='left', on='codigo_ibge_data')

In [39]:
codigo_ibge_data_abertura = codigo_ibge_data_abertura.drop(['codigo_ibge_data','regiao','estado','codigo_ibge_ajustado','data_covid','populacaoTCU2019','casosNovos','obitosNovos'], axis=1) #eliminação das colunas desnecessárias

In [40]:
codigo_ibge_data_abertura.columns = ['casosAcumulado_corte','obitosAcumulado_corte','casos_acumulados_porcentagem_corte','obitos_acumulados_porcentagem_corte']

In [41]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_abertura], axis=1)

In [42]:
idh_municipios = pd.read_csv('//content/drive/MyDrive//TCC - PUC/IDH por Município - FINAL COM CÓDIGO IBGE.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [43]:
idh_municipios = idh_municipios.drop(['Territorialidade','Posicao IDHM','Posicao IDHM Educacao','Posicao IDHM Longevidade','Posicao IDHM Renda'], axis=1) #eliminação das colunas desnecessárias

In [44]:
idh_municipios.columns = ['codigo_ibge', 'IDH DO MUNICIPIO', 'IDHM EDUCACAO DO MUNICIPIO', 'IDHM LONGEVIDADE DO MUNICIPIO','IDHM RENDA DO MUNICIPIO']

In [45]:
cnpj_final = pd.merge(cnpj_final, idh_municipios, how='left', on='codigo_ibge')

In [46]:
pib_municipios = pd.read_csv('//content/drive/MyDrive/TCC - PUC/PIB dos Municípios - base de dados 2010-2018 - AJUSTADO.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [47]:
pib_municipios = pib_municipios.drop(['Sigla da Unidade da Federação','Nome do Município','Ano'], axis=1) #eliminação das colunas desnecessárias

In [48]:
pib_municipios.columns = ['codigo_ibge', 'PIB', 'PIB PER CAPITA']

In [49]:
cnpj_final = pd.merge(cnpj_final, pib_municipios, how='left', on='codigo_ibge')

In [50]:
cnae = pd.read_csv('//content/drive/MyDrive/TCC - PUC/CLASSIFICACAO CNAE POR SECAO.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)

In [51]:
cnae.columns = ['cnae_divisao', 'cnae_secao','cnae_descricao']

In [52]:
cnae_divisao = cnpj_final['cnae_principal'].map(lambda x: str(x)[:-5])

In [53]:
cnae_divisao = cnae_divisao.astype(int)

In [54]:
cnpj_final = pd.concat([cnpj_final, cnae_divisao], axis=1)

In [55]:
cnpj_final.columns = ['cnpj_basico','cnpj_ordem','cnpj_dv','matriz_filial','codigo_situacao_cadastral','data_situacao_cadastral','motivo_situacao_cadastral','data_inicio_atividade','cnae_principal','codigo_municipio_siafi','capital_social','descricao_municipio','codigo_ibge','codigo_ibge_ajustado','codigo_ibge_data_fechamento','codigo_ibge_data_abertura','regiao','estado','populacaoTCU2019','casosAcumulado_covid','obitosAcumulado_covid','casos_acumulados_porcentagem_covid','obitos_acumulados_porcentagem_covid','casosAcumulado_corte','obitosAcumulado_corte','casos_acumulados_porcentagem_corte','obitos_acumulados_porcentagem_corte','IDH DO MUNICIPIO','IDHM EDUCACAO DO MUNICIPIO','IDHM LONGEVIDADE DO MUNICIPIO','IDHM RENDA DO MUNICIPIO','PIB','PIB PER CAPITA','cnae_divisao']

In [56]:
secao_condicao = [
    (cnpj_final['cnae_divisao'] >= 1) & (cnpj_final['cnae_divisao'] <=3),
    (cnpj_final['cnae_divisao'] >= 5) & (cnpj_final['cnae_divisao'] <=9),
    (cnpj_final['cnae_divisao'] >= 10) & (cnpj_final['cnae_divisao'] <=33),
    (cnpj_final['cnae_divisao'] == 35),
    (cnpj_final['cnae_divisao'] >= 36) & (cnpj_final['cnae_divisao'] <=39),
    (cnpj_final['cnae_divisao'] >= 41) & (cnpj_final['cnae_divisao'] <=43),
    (cnpj_final['cnae_divisao'] >= 45) & (cnpj_final['cnae_divisao'] <=47),
    (cnpj_final['cnae_divisao'] >= 49) & (cnpj_final['cnae_divisao'] <=53),
    (cnpj_final['cnae_divisao'] >= 55) & (cnpj_final['cnae_divisao'] <=56),
    (cnpj_final['cnae_divisao'] >= 58) & (cnpj_final['cnae_divisao'] <=63),
    (cnpj_final['cnae_divisao'] >= 64) & (cnpj_final['cnae_divisao'] <=66),
    (cnpj_final['cnae_divisao'] == 68),
    (cnpj_final['cnae_divisao'] >= 69) & (cnpj_final['cnae_divisao'] <=75),
    (cnpj_final['cnae_divisao'] >= 77) & (cnpj_final['cnae_divisao'] <=82),
    (cnpj_final['cnae_divisao'] == 84),
    (cnpj_final['cnae_divisao'] == 85),
    (cnpj_final['cnae_divisao'] >= 86) & (cnpj_final['cnae_divisao'] <=88),
    (cnpj_final['cnae_divisao'] >= 90) & (cnpj_final['cnae_divisao'] <=93),
    (cnpj_final['cnae_divisao'] >= 94) & (cnpj_final['cnae_divisao'] <=96),
    (cnpj_final['cnae_divisao'] == 97),
    (cnpj_final['cnae_divisao'] == 99)]
cnae_secao = ['a', 'b', 'c','d', 'e', 'f','g', 'h', 'i','j', 'k', 'l','m', 'n', 'o','p', 'q', 'r','s', 't', 'u']
cnpj_final['cnae_secao'] = np.select(secao_condicao, cnae_secao)

In [57]:
cnpj_final['PIB'] = cnpj_final['PIB'].str.replace('.', '')
cnpj_final['PIB PER CAPITA'] = cnpj_final['PIB PER CAPITA'].str.replace('.', '')

In [58]:
cnpj_final['capital_social'] = cnpj_final['capital_social'].str.replace(',', '.')
cnpj_final['IDH DO MUNICIPIO'] = cnpj_final['IDH DO MUNICIPIO'].str.replace(',', '.')
cnpj_final['IDHM EDUCACAO DO MUNICIPIO'] = cnpj_final['IDHM EDUCACAO DO MUNICIPIO'].str.replace(',', '.')
cnpj_final['IDHM LONGEVIDADE DO MUNICIPIO'] = cnpj_final['IDHM LONGEVIDADE DO MUNICIPIO'].str.replace(',', '.')
cnpj_final['IDHM RENDA DO MUNICIPIO'] = cnpj_final['IDHM RENDA DO MUNICIPIO'].str.replace(',', '.')
cnpj_final['PIB'] = cnpj_final['PIB'].str.replace(',', '.')
cnpj_final['PIB PER CAPITA'] = cnpj_final['PIB PER CAPITA'].str.replace(',', '.')


In [59]:
cnpj_final[["capital_social", "IDH DO MUNICIPIO","IDHM EDUCACAO DO MUNICIPIO","IDHM LONGEVIDADE DO MUNICIPIO","IDHM RENDA DO MUNICIPIO","PIB","PIB PER CAPITA"]] = cnpj_final[["capital_social", "IDH DO MUNICIPIO","IDHM EDUCACAO DO MUNICIPIO","IDHM LONGEVIDADE DO MUNICIPIO","IDHM RENDA DO MUNICIPIO","PIB","PIB PER CAPITA"]].apply(pd.to_numeric)

In [60]:
cnpj_final = cnpj_final[cnpj_final.codigo_municipio_siafi != 9707]  #EMPRESAS DO EXTERIOR

In [61]:
cnpj_final = cnpj_final.drop(['codigo_ibge_ajustado','codigo_ibge_data_fechamento','codigo_ibge_data_abertura'], axis=1) #eliminação das colunas desnecessárias

In [62]:
cnpj_final.fillna(0, inplace = True)

In [63]:
cnpj_final.to_csv('/content/drive/MyDrive/TCC - PUC/dataset_final.csv', index=False, sep=';', encoding='ISO-8859-1')